In [4]:
import pandas as pd
import numpy as np
import json
import ast
from ast import literal_eval
from datetime import datetime

df = pd.read_csv('movies_dataset.csv',sep=",")
#como el none nos estaba dando problemas lo eliminamos con esta funcion
def parse_dict_str(value):
    if pd.isna(value):
        return None
    else:
        return ast.literal_eval(value)
def parse_dict_str2(value):
    if pd.isna(value):
        return None
    else:
        genres = ast.literal_eval(value)
        return ','.join([g['name'] for g in genres])
def parse_dict_str3(value):
    if pd.isna(value):
        return None
    else:
        languages = ast.literal_eval(value)
        return ','.join([c['name'] for c in languages])     
def parse_dict_str4(value):
    if pd.isna(value):
        return None
    else:
        try:
            companies = ast.literal_eval(value)
            if isinstance(companies, list):
                return ','.join([c['name'] for c in companies])
            else:
                return 'Unknown'
        except (ValueError, TypeError):
            # If the input value is not a valid dictionary or list, split it on spaces and process each word separately
            words = value.split()
            return ','.join(words)
        
def extract_year(date_string):
    if pd.isna(date_string):
        return None
    try:
        date_object = datetime.strptime(date_string, '%Y-%m-%d')
        return date_object.year
    except ValueError: 
        return None
dfnormalizedprocountri = pd.json_normalize(df['production_countries']).fillna('')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = df['revenue'].astype(float)
df['return'] = df['revenue'].div(df['budget'])
df['year'] = df['release_date'].apply(extract_year)
df['belongs_to_collection'] = df['belongs_to_collection'].apply(parse_dict_str)
dfnormalizedbelongs = pd.json_normalize(df['belongs_to_collection']).fillna('')
#dfnormalizedproduction = pd.json_normalize(df['production_companies']).fillna('')
df['production_countries'] = df['production_countries'].apply(parse_dict_str4)
df = pd.concat([df, dfnormalizedbelongs['name']], axis=1)
df['genres'] = df['genres'].apply(parse_dict_str2)
df['spoken_languages'] = df['spoken_languages'].apply(parse_dict_str3)
#df['production_companies'] = df['production_companies'].apply(parse_dict_str)
df['production_companies'] = df['production_companies'].apply(parse_dict_str4)
#dfnormalizedgenres = pd.json_normalize(df['name']).fillna('')
#df = pd.concat([df, dfnormalizedgenres], axis=1)
#df['year'] = df['year'].replace([np.inf, -np.inf], ^np.nan)
df['vote_count'] = df['vote_count'].fillna(0).astype(int)
df['year'] = df['year'].fillna(0).astype(int)
df['return'].fillna(0)
df = df.drop(['belongs_to_collection','video','imdb_id','adult','original_title','poster_path','homepage'], axis=1)
#df[['adult', 'name', 'budget', 'revenue', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'release_date','year']]	
column = df.pop('name')
df.insert(0,'name', column)
info = input('ingrese el titulo de la pelicula')
vote_count = df.loc[df['title'] == info , 'vote_count'].values[0]
score = df.loc[df['title'] == info, 'popularity'].values[0]
year = df.loc[df['title'] == info, 'year'].values[0]
vote_average = df.loc[df['title'] == info, 'vote_average'].values[0]
if vote_count > 2000:
    print('La película', info, 'fue estrenada en el año.', year, 'La misma cuenta con un total de', vote_count,'valoraciones, con un promedio de', vote_average)
else:
    print('la pelicula que busca no cumple con la condicion de mas de 2000 votos por ende no se devolvera ningun valor') 
print('el año fue :', year, 'y la puntuacion fue', score)
df['year']
#df.to_csv('movies_dataset.csv', index=False)

C:\Users\carlo\AppData\Local\Temp\ipykernel_98660\1551005435.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv',sep=",")


KeyError: 'belongs_to_collection'

In [29]:
import pandas as pd
df2 = pd.read_csv('credits.csv',sep = ',')
#-----------------------------------------------
#Desanidaremos la columna "cast"
#-----------------------------------------------
# Reemplazamos NaN por una lista compuesta por un diccionario vacío
df2['cast'] = df2['cast'].apply(lambda x: [] if pd.isna(x) else x)
# Utilizamos una función personalizada que verifique si el valor es una cadena y 
# luego intente convertirla en una lista para finalmente extraer los nombres del cast

def extract_cast_names(value):
    if isinstance(value, str):
        try:
            cast_list = ast.literal_eval(value)
            return [d.get('name') for d in cast_list]
        except (ValueError, SyntaxError):
            return None
    elif isinstance(value, list):
        return [d.get('name') for d in value]
    else:
        return None
#Creamos la columna "cast_names" compuesta por los valores de la clave
#"name" de los diccionarios de las listas de la columna "cast"
df2['cast_names'] = df2['cast'].apply(extract_cast_names)
#Reemplazamos las listas vacías por None
df2['cast_names'] = df2['cast_names'].replace([], None)
#Eliminamos la columna "cast" original
df2 = df2.drop('cast', axis=1)
#-----------------------------------------------
#Desanidaremos la columna "crew"
#-----------------------------------------------
# Reemplazamos NaN por una lista compuesta por un diccionario vacío
df2['crew'] = df2['crew'].apply(lambda x: [] if pd.isna(x) else x)
# Utilizamos una función personalizada que verifique si el valor es una cadena y 
#luego intente convertirla en una lista para finalmente extraer los nombres del crew

def extract_crew_names(value):
    if isinstance(value, str):
        try:
            crew_list = ast.literal_eval(value)
            return [d.get('name') for d in crew_list]
        except (ValueError, SyntaxError):
            return None
    elif isinstance(value, list):
        return [d.get('name') for d in value]
    else:
        return None

#Creamos la columna "crew_names" compuesta por los valores de la clave
#"name" de los diccionarios de las listas de la columna "crew"

df2['crew_names'] = df2['crew'].apply(extract_crew_names)

#Reemplazamos las listas vacías por None
df2['crew_names'] = df2['crew_names'].replace([], None)

#Eliminamos la columna "crew" original
df2 = df2.drop('crew', axis=1)

#df['directors'] = df['crew'].apply(lambda x: [i['name'] for i in eval(x) if i['job'] == 'Director'])
 # Imprimir la lista de directores
#print(df['directors'])
#Creamos la columna "cast_names" compuesta por los valores de la clave
#"name" de los diccionarios de las listas de la columna "cast"
#df2['cast_names'] = df2['cast'].apply(extract_cast_names)
#Reemplazamos las listas vacías por None
#df2['cast_names'] = df2['cast_names'].replace([], None)

#df = pd.read_csv('movies_dataset.csv')
 # read the second CSV file into a DataFrame
#df2 = pd.read_csv('credits.csv')
 # concatenate the two DataFrames horizontally
#result = pd.concat([df, df2], axis=1)
#df = df.sort_values(by=['id'])
#df2 = df2.sort_values(by=['id.1'])
 # save the concatenated DataFrame to a new CSV file
result.to_csv('final.csv', index=False)

In [11]:
import pandas as pd
import numpy as np
actor_revenue = 0
df = pd.read_csv('final.csv')
df['id.1'] = df['id']
df.to_csv('final.csv', index=False)

C:\Users\carlo\AppData\Local\Temp\ipykernel_14568\3395413607.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final.csv')


In [4]:
import pandas as pd
import numpy as np
actor_revenue = 0
df = pd.read_csv('final.csv')
#df['id.1'] = df['id']
info = input('Ingrese el nombre del actor: ')
actor_rows = df[df['cast_names'].str.contains(info)]
actor_revenue = df[df['cast_names'] == info].groupby('cast_names')['revenue'].sum()
movies_count = len(actor_rows)
print(movies_count)
print(actor_revenue)
print(actor_revenue/movies_count)


C:\Users\carlo\AppData\Local\Temp\ipykernel_14568\3731317033.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final.csv')


,name,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,tagline,title,vote_average,vote_count,return,year,id.1,cast_names,crew_names,directors
44976,NaN,0.0,Drama,163202,zh,"Sega's children, born into post-war Chinese ru...",0.061654,Long Shong Pictures,Taiwan,1994-01-01,...,NaN,A Borrowed Life,5.0,1.0,NaN,1994.0,163202,"['Valerio Mastandrea', 'Bérénice Bejo', 'Guido...","['Beppe Caschetto', 'Marco Bellocchio', 'Marco...",['Marco Bellocchio']
44977,NaN,6000000.0,"Action,Thriller",363925,pt,A group of honest police is sent to a country ...,2.514515,"Globo filmes,Querosene Filmes,TC Filmes",Brazil,2015-10-07,...,NaN,Operações Especiais,6.8,31.0,0.0,2015.0,363925,"['Alex Meneses', 'Faizon Love', 'Russell Peter...","['Brad Epstein', 'Billiam Coronel', 'Brad Epst...",['Brad Epstein']
44978,NaN,0.0,NaN,398295,pt,NaN,0.000331,NaN,NaN,NaN,...,NaN,Mundo Cão,0.0,0.0,NaN,0.0,398295,"['Anneli Sauli', 'Saulo Haarla', 'Tauno Palo',...",['T.J. Särkkä'],['T.J. Särkkä']
44979,NaN,0.0,NaN,132320,es,"David Alfaro Siqueiros, recognized Mexican mur...",0.529543,NaN,NaN,2011-08-19,...,NaN,The Mural of Siqueiros,5.0,2.0,NaN,2011.0,132320,"['Steve Boyle', 'Nic Rhind', 'Robert Hobbs', '...","['Neill Blomkamp', 'Neill Blomkamp', 'Thomas S...",['Neill Blomkamp']
44980,Batman Beyond Collection,0.0,"Animation,Action,Family",64202,en,"Fueled by remorse and vengeance, a high school...",2.363167,"DC Comics,Warner Bros. Family Entertainment",NaN,1999-01-10,...,"A New Hero, For A New Era",Batman Beyond: The Movie,7.2,41.0,NaN,1999.0,64202,"['Minna Haapkylä', 'Petteri Summanen', 'Laura ...","['Sakari Salli', 'Jukka Nykänen', 'Joona Tena']",['Joona Tena']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Leila Hatami', 'Kourosh Tahami', 'Elham Korda']","['Hamid Nematollah', 'Hamid Nematollah', 'Fars...",['Hamid Nematollah']
45472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Angel Aquino', 'Perry Dizon', 'Hazel Orencio...","['Lav Diaz', 'Lav Diaz', 'Dante Perez', 'Lav D...",['Lav Diaz']
45473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Erika Eleniak', 'Adam Baldwin', 'Julie du Pa...","['Mark L. Lester', 'C. Courtney Joyner', 'Jeff...",['Mark L. Lester']
45474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Iwan Mosschuchin', 'Nathalie Lissenko', 'Pav...","['Yakov Protazanov', 'Joseph N. Ermolieff']",['Yakov Protazanov']


In [3]:
import pandas as pd

df = pd.read_csv('final.csv')
info = input('Ingrese el nombre del director: ')

director_films = df[df['directors'].str.contains(info)]
director_movies = df[df['cast_names'] == info]
director_titles = df[df['directors'].str.contains(info, na=False)]['title'].tolist()
director_return = df[df['directors'].str.contains(info, na=False)]['return'].sum()
for movie in director_titles:
    revenue = df.loc[df['title'] == movie, 'revenue'].values[0]
    print('las peliuclas y sus ganancias : ' , movie, revenue)
#movies_returns = director_titles
directors_count = len(director_films)
print('el numero de pelicual que a dirijido fue :', directors_count)
print('sus peliculas fueron ', director_titles)
print(' sus ganancias en total: ', director_return)
#print(director_movies_count)


C:\Users\carlo\AppData\Local\Temp\ipykernel_2964\3756914515.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final.csv')


las peliuclas y sus ganancias :  Toy Story 373554033.0
las peliuclas y sus ganancias :  Jumanji 262797249.0
las peliuclas y sus ganancias :  Grumpier Old Men 0.0
las peliuclas y sus ganancias :  Waiting to Exhale 81452156.0
las peliuclas y sus ganancias :  Father of the Bride Part II 76578911.0
las peliuclas y sus ganancias :  Heat 187436818.0
las peliuclas y sus ganancias :  Sabrina 0.0
las peliuclas y sus ganancias :  Tom and Huck 0.0
las peliuclas y sus ganancias :  Sudden Death 64350171.0
las peliuclas y sus ganancias :  GoldenEye 352194034.0
las peliuclas y sus ganancias :  The American President 107879496.0
las peliuclas y sus ganancias :  Dracula: Dead and Loving It 0.0
las peliuclas y sus ganancias :  Balto 11348324.0
las peliuclas y sus ganancias :  Nixon 13681765.0
las peliuclas y sus ganancias :  Cutthroat Island 10017322.0
las peliuclas y sus ganancias :  Casino 116112375.0
las peliuclas y sus ganancias :  Sense and Sensibility 135000000.0
las peliuclas y sus ganancias :  F

las peliuclas y sus ganancias :  Inside Daisy Clover 0.0
las peliuclas y sus ganancias :  Gamera vs. Gyaos 0.0
las peliuclas y sus ganancias :  Over the Edge 0.0
las peliuclas y sus ganancias :  Roads to Koktebel 0.0
las peliuclas y sus ganancias :  Swamp Women 0.0
las peliuclas y sus ganancias :  8MM 2 0.0
las peliuclas y sus ganancias :  On Probation 0.0
las peliuclas y sus ganancias :  Bombón: The Dog 0.0
las peliuclas y sus ganancias :  The Bird People in China 0.0
las peliuclas y sus ganancias :  Thoughtcrimes 0.0
las peliuclas y sus ganancias :  Water 10422387.0
las peliuclas y sus ganancias :  The Temptations 0.0
las peliuclas y sus ganancias :  Sophie Scholl: The Final Days 0.0
las peliuclas y sus ganancias :  The Big House 0.0
las peliuclas y sus ganancias :  Uuno Turhapuro armeijan leivissä 0.0
las peliuclas y sus ganancias :  Matti: Hell Is for Heroes 0.0
las peliuclas y sus ganancias :  The World's Fastest Indian 0.0
las peliuclas y sus ganancias :  As Tears Go By 0.0
las p

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv('final.csv')

df.tail(500)

C:\Users\carlo\AppData\Local\Temp\ipykernel_14568\3833369242.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final.csv')


,name,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,tagline,title,vote_average,vote_count,return,year,id.1,cast_names,crew_names,directors
44879,Totò Collection,0.0,Comedy,63179,it,NaN,0.566432,Titanus,Italy,1950-11-30,...,NaN,Totò Sceicco,6.5,8.0,NaN,1950.0,63179,"['Aldo Baglio', 'Giovanni Storti', 'Giacomo Po...","['Massimo Venier', 'Aldo Baglio', 'Giovanni St...","['Massimo Venier', 'Aldo Baglio', 'Giovanni St..."
44880,NaN,0.0,Drama,398924,fr,"In occupied France, Maurice and Joseph, two yo...",12.389862,"TF1 Films Productions,Forecast Pictures,Quad F...","Czech Republic,France,Canada",2017-01-18,...,Home Is Where Family Is,A Bag of Marbles,7.8,100.0,NaN,2017.0,398924,"['Arnoldo Foà', 'Bebo Storti', 'Giorgio Gherar...","['Fabrizio Lucci', 'Esmeralda Calabria', 'Lili...",['Paolo Costella']
44881,NaN,0.0,NaN,175910,en,A documentary about behind the scenes and what...,0.040398,NaN,NaN,2002-09-06,...,Documentary,7 Days in September,8.0,1.0,NaN,2002.0,175910,"['Aldo Baglio', 'Giovanni Storti', 'Giacomo Po...","['Massimo Venier', 'Aldo Baglio', 'Giacomo Por...","['Massimo Venier', 'Aldo Baglio', 'Giacomo Por..."
44882,NaN,0.0,"Science Fiction,Thriller,Drama,Action",32228,en,A physicist discovers a briefcase containing p...,1.05306,NaN,"Canada,United States of America",2004-01-01,...,NaN,5ive Days to Midnight,6.8,18.0,NaN,2004.0,32228,[],['Billy Wilder'],['Billy Wilder']
44883,NaN,0.0,Documentary,436450,en,"Like it or not, porn is here and it is harmful...",0.279521,"Expressive Artists,Time & Tide Productions",United States of America,2017-02-03,...,NaN,Addicted to Porn: Chasing the Cardboard Butterfly,5.0,3.0,NaN,2017.0,436450,"['Liev Schreiber', 'Naomi Watts', 'Elisabeth M...","['Billy Hopkins', 'Richard Comeau', 'Liev Schr...",['Philippe Falardeau']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45374,NaN,0.0,"Drama,Action,Romance",30840,en,"Yet another version of the classic epic, with ...",5.683753,"Westdeutscher Rundfunk (WDR),Working Title Fil...","Canada,Germany,United Kingdom,United States of...",1991-05-13,...,NaN,Robin Hood,5.7,26.0,NaN,1991.0,30840,['Georges Méliès'],['Georges Méliès'],['Georges Méliès']
45375,NaN,0.0,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,Sine Olivia,Philippines,2011-11-17,...,NaN,Century of Birthing,9.0,3.0,NaN,2011.0,111109,[],"['Robert Fulton', 'Robert Gardner', 'Robert Ga...",['Robert Gardner']
45376,NaN,0.0,"Action,Drama,Thriller",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,American World Pictures,United States of America,2003-08-01,...,A deadly game of wits.,Betrayal,3.8,6.0,NaN,2003.0,67758,"['Sridevi Kapoor', 'Sajal Ali', 'Akshaye Khann...","['A. R. Rahman', 'Boney Kapoor', 'Mukesh Talre...",['Ravi Udyawar']
45377,NaN,0.0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,Yermoliev,Russia,1917-10-21,...,NaN,Satan Triumphant,0.0,0.0,NaN,1917.0,227506,"['Markie Adams', 'Roberto Aguire', 'Tina Arnin...","['Shanra J. Kehl', 'Shanra J. Kehl']",['Shanra J. Kehl']


In [8]:
import pandas as pd
df = pd.read_csv('final.csv')

info = input('ingrese el titulo de la pelicula')
vote_count = df.loc[df['title'] == info , 'vote_count'].values[0]
score = df.loc[df['title'] == info, 'popularity'].values[0]
year = df.loc[df['title'] == info, 'year'].values[0]

vote_average = df.loc[df['title'] == info, 'vote_average'].values[0]
if vote_count > 2000:
    print('La película', info, 'fue estrenada en el año.', year, 'La misma cuenta con un total de', vote_count,'valoraciones, con un promedio de', vote_average)
else:
    print('la pelicula que busca no cumple con la condicion de mas de 2000 votos por ende no se devolvera ningun valor') 
print('el año fue :', year, 'y la puntuacion fue', score)


C:\Users\carlo\AppData\Local\Temp\ipykernel_98660\1254898615.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final.csv')


La película Toy Story fue estrenada en el año. 1995.0 La misma cuenta con un total de 5415.0 valoraciones, con un promedio de 7.7
el año fue : 1995.0 y la puntuacion fue 21.946943


0        1995.0
1        1995.0
2        1995.0
3        1995.0
4        1995.0
          ...  
45471       NaN
45472       NaN
45473       NaN
45474       NaN
45475       NaN
Name: year, Length: 45476, dtype: float64

In [2]:
import pandas as pd 
from datetime import datetime
df = pd.read_csv('final.csv')
def extract_month(date_string):
    if pd.isna(date_string):
        return None
    try:
        date_object = datetime.strptime(date_string, '%Y-%m-%d')
        return date_object.month
    except ValueError: 
        return None
def extract_day(date_string):
    if pd.isna(date_string):
        return None
    try:
        date_object = datetime.strptime(date_string, '%Y-%m-%d')
        return date_object.weekday() + 1
    except ValueError: 
        return None

meses = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto':8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
}
dias = {
    'lunes': 1,
    'martes': 2,
    'miércoles': 3,
    'jueves': 4,
    'viernes': 5,
    'sábado': 6,
    'sabado' : 6,
    'domingo': 7
}

info = input('Ingrese el mes en español: ').lower()
if info in meses:
    mes = meses[info]   
else:
    print('El mes ingresado no es válido.')

info2 = input('Ingrese el dia en español: ').lower()
if info2 in dias:
    dia = dias[info2]
else:
    print('El dia ingresado no es válido.')   


df['month'] = df['release_date'].apply(extract_month)
df['day'] = df['release_date'].apply(extract_day)

month = df.loc[df['month'] == mes, 'month'].count()
day = df.loc[df['day'] == dia, 'day'].shape[0]
print(month)
print(day)


C:\Users\carlo\AppData\Local\Temp\ipykernel_12556\1281999807.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final.csv')


3786
3503


In [9]:

df = df.dropna(subset=['release_date'])
 
df = df.reset_index(drop=True)
 
df.to_csv('final.csv', index=False)

In [12]:


null_counts = df.isnull().sum()
print(null_counts)

name                    40891
budget                      3
genres                   2384
id                          0
original_language          11
overview                  941
popularity                  2
production_companies    11796
production_countries     6211
release_date                0
revenue                     3
runtime                   249
spoken_languages         3894
status                     83
tagline                 24981
title                       3
vote_average                3
vote_count                  0
return                  34467
year                        0
id.1                        0
cast_names                  0
crew_names                  0
directors                   0
dtype: int64
